In [1]:
import easyocr
import cv2

testFile = '/media/chowder/abhiksFiles/personalData/wyzeCams/roomCamBy3dp/20201230/10/20.mp4'

def prepFramesFromFile(fil):
    cap = cv2.VideoCapture(fil)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        x1, y1 = 1350, 1000  # Starting coordinates
        x2, y2 = 1900, 1080  # Ending coordinates
        cropped_frame = frame[y1:y2, x1:x2]
        image_resized = cv2.resize(cropped_frame, (cropped_frame.shape[1] // 2, cropped_frame.shape[0] // 2), interpolation=cv2.INTER_LINEAR)
        frames.append(image_resized)
    return enumerate(frames)

In [2]:
# 18s for 901 full timesamps no resizing
# 15s for 901 full timesamps 1/2 resizing
def doOCR(reader, frames):
    return enumerate([reader.readtext(frame) for frame in frames])

In [3]:
# only keep didgits and add a T after the first 8 didgits
def OCRtoISOString(OCRs):
    timeStrings = []
    for result in results:
        justDidgits = ""
        for (bbox, text, prob) in OCRs:
            justDidgits = justDidgits + ''.join([char for char in text if char.isdigit()])
            # print(text)
        timeStrings.append(justDidgits[:8] + "T" + justDidgits[8:])


In [5]:
# 60/sec processing 900 frames per video 34k videos about 6 days of processing

# getting microsecond timestamp from timestamp

# lets get a dataset of the strings of a few videos and test

In [6]:
import os
def getFileList(daysPath):
    fileList = []
    for dayFolder in os.listdir(daysPath):
        hourFoldersPath = daysPath + dayFolder + '/'
        for hourFolder in os.listdir(hourFoldersPath):
            for minuteFile in os.listdir(hourFoldersPath + hourFolder + '/'):
                # we are looking to build an iso timestring from this info
                fileTime = dayFolder + "T" + hourFolder + minuteFile.split('.')[0] + "00"
                fileList.append(fileTime)
    return fileList
def fileLocation(daysPath, fileListName):
    return daysPath + fileListName[:8] + '/' + fileListName[9:11] + '/' + fileListName[12:14] + ".mp4"
daysPath = '/media/chowder/abhiksFiles/personalData/wyzeCams/roomCamBy3dp/'
fl = getFileList(daysPath)

In [7]:
reader = easyocr.Reader(['en'], gpu=True)

In [8]:
def prepFramesFromFile(fil):
    cap = cv2.VideoCapture(fil)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        x1, y1 = 1350, 1000  # Starting coordinates
        x2, y2 = 1900, 1080  # Ending coordinates
        cropped_frame = frame[y1:y2, x1:x2]
        image_resized = cv2.resize(cropped_frame, (cropped_frame.shape[1] // 2, cropped_frame.shape[0] // 2), interpolation=cv2.INTER_LINEAR)
        frames.append(image_resized)
    return frames

In [9]:
def doOCR(reader, frames):
    return [reader.readtext(frame) for frame in frames]

In [10]:
def OCRtoISOString(OCRs):
    timeStrings = []
    for result in OCRs:
        justDidgits = ""
        for (bbox, text, prob) in result:
            justDidgits = justDidgits + ''.join([char for char in text if char.isdigit()])
            # print(text)
        timeStrings.append(justDidgits[:4] + # "-" +
                            justDidgits[4:6] + # "-" +
                            justDidgits[6:8] + "T" + justDidgits[8:])
    return  timeStrings

In [19]:
#overall structure
#(fileListName, frame number, OCRtoISO)
# OCRTimes = []
for fileLisName in fl[10:50]:
    fileLoc = fileLocation(daysPath, fileLisName)
    frames = prepFramesFromFile(fileLoc)
    OCRs = doOCR(reader, frames)
    ISOstrings = OCRtoISOString(OCRs)
    OCRTimes += [[fileLisName, frameNum, ISOstr] for frameNum, ISOstr in enumerate(ISOstrings)]

In [12]:
import pandas as pd
columnNames = ['fileName', 'frameNumber', 'OCRSecond']
ocrDf = pd.DataFrame(OCRTimes, columns=columnNames)
ocrDf.head()

,fileName,frameNumber,OCRSecond
0,20201230T102000,0,20201230T100001
1,20201230T102000,1,20201230T100001
2,20201230T102000,2,20201230T100001
3,20201230T102000,3,20201230T100001
4,20201230T102000,4,20201230T100001


In [14]:
ocrDf.shape

(6277, 3)

In [18]:
ocrDf['OCRSecond'].unique().shape

(365,)

In [27]:
ocrDf = ocrDf.drop('value', axis=1)

In [29]:
# Count the number of occurrences of each unique string
ocrDf['count'] = ocrDf.groupby('OCRSecond')['OCRSecond'].transform('count')

# Create a new column with the value 1 / count
ocrDf['secOffset'] = (1 / ocrDf['count']) * ocrDf['frameNumber']
ocrDf.head(100)

,fileName,frameNumber,OCRSecond,count,secOffset
0,20201230T102000,0,20201230T100001,12,0.000000
1,20201230T102000,1,20201230T100001,12,0.083333
2,20201230T102000,2,20201230T100001,12,0.166667
3,20201230T102000,3,20201230T100001,12,0.250000
4,20201230T102000,4,20201230T100001,12,0.333333
...,...,...,...,...,...
95,20201230T102000,95,20201230T100007,30,3.166667
96,20201230T102000,96,20201230T100007,30,3.200000
97,20201230T102000,97,20201230T100007,30,3.233333
98,20201230T102000,98,20201230T100008,30,3.266667


In [31]:
ocrDf['count'].unique()

array([12, 34, 30, 24, 36, 28, 18, 14, 15, 11, 16,  5,  2, 17,  7, 10,  3,
       13])

In [35]:
ocrDf['count'].value_counts()

count
15    3630
30    1470
18     306
12     252
14     196
36     144
24      96
17      51
34      34
28      28
16      16
13      13
11      11
10      10
7        7
3        6
5        5
2        2
Name: count, dtype: int64

In [ ]:
# now what would I like to do?
# want to tell the approximate fps
# count the number of elements for each key value'

#use the fps to get an approximate microsecond offset per frame

def calcApproxMicrosecond(ocrDf):
    ocrDf
    pass